<a href="https://colab.research.google.com/github/harnalashok/deeplearning-sequences/blob/main/network_connect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cdlib

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
#%reset -f

# 1.0 Call libraries
import pandas as pd
import numpy as np
import networkx as nx

# 1.01
import joblib
from collections import defaultdict

# 1.02
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# 1.08 Misc
import os,gc , time,  glob
from os.path import exists
import feather
import re
import ast


In [19]:
# Paths:
modulePath = "/gdrive/MyDrive/Colab_data_files/talkingData/03112022/"
dataPath = "/gdrive/MyDrive/Colab_data_files/skipgram/"
modelsPath = "/gdrive/MyDrive/Colab_data_files/skipgram/modelspath/"
pathToStoreProgress = "/gdrive/MyDrive/Colab_data_files/skipgram/pathToStoreProgress/"
mapdictPath = "/gdrive/MyDrive/Colab_data_files/skipgram/mapdictpath/"

In [5]:
# 1.09 Home made modules
os.chdir(modulePath)
import network_features
from network_features import *
import utils
from utils import *

import importlib; importlib.reload(network_features)
import importlib; importlib.reload(utils)

Note: to be able to use all crisp methods, you need to install some additional packages:  {'infomap', 'wurlitzer', 'karateclub', 'leidenalg', 'graph_tool'}
Note: to be able to use all overlapping methods, you need to install some additional packages:  {'karateclub', 'ASLPAw'}
Note: to be able to use all bipartite methods, you need to install some additional packages:  {'infomap', 'wurlitzer', 'leidenalg'}


<module 'utils' from '/gdrive/MyDrive/Colab_data_files/talkingData/03112022/utils.py'>

In [32]:
# Cleaning up
!rm -r -f /gdrive/MyDrive/Colab_data_files/skipgram/modelspath/
!rm -r -f /gdrive/MyDrive/Colab_data_files/skipgram/pathToStoreProgress/
!rm -r -f /gdrive/MyDrive/Colab_data_files/skipgram/mapdictpath/

In [33]:
!rm -r -f /gdrive/MyDrive/Colab_data_files/skipgram/vectors.tsv
!rm -r -f /gdrive/MyDrive/Colab_data_files/skipgram/metadata.tsv

In [34]:

!mkdir /gdrive/MyDrive/Colab_data_files/skipgram/modelspath/
!mkdir /gdrive/MyDrive/Colab_data_files/skipgram/pathToStoreProgress/
!mkdir /gdrive/MyDrive/Colab_data_files/skipgram/mapdictpath/

In [ ]:
# 8.0 Read back pkl file and dictionary:
os.chdir(dataPath)
train = pd.read_pickle("seq.pkl")
train.head()

In [ ]:
# Read dictionary
os.chdir(dataPath)
filehandler = open("word_index.txt", 'r')
word2index = filehandler.read()
word2index

In [37]:
org_columns = train.columns
cat_cols = ['a', 'b']

In [38]:
# 6.1 Create new columns (combinations of columns) in train
train = utils.create_cat_pairwise_cols(train , cat_cols)

In [39]:
train.head()

,a,b,a_p_b
0,5,1554,5_1554
1,7,261,7_261
2,30,2621,30_2621
3,320,74,320_74
4,1,3397,1_3397


In [40]:
# 6.11 The train columns are:
current_train_columns = train.columns
# 6.12 New columns created. These columns must be latter dropped
new_cols = list(set(current_train_columns).difference(set(org_columns)))
# 6.13 Revised cat cols (ie excluding hour etc)
cat_cols.extend(new_cols)
len(cat_cols)   # 15
cat_cols

['a', 'b', 'a_p_b']

In [41]:
# 8.1 Get all possibe permutation of our columns
#     Also decide which all centrality measures to cal
deg = 1    # Calculate degree centrality
eig = 1    # Calculate eigenvector centrality
pr =  1    # Calculate pagerank centrality
clu = 0    # Clustering coeff centrality
nf = None # nx.katz_centrality_numpy
cMeasures = [deg,eig,pr,clu, nf]

In [42]:
single_permute_list = utils.permute_colList(cat_cols,cMeasures)
single_permute_list

[('a', 'b', [1, 1, 1, 0, None]),
 ('a', 'a_p_b', [1, 1, 1, 0, None]),
 ('b', 'a', [1, 1, 1, 0, None]),
 ('b', 'a_p_b', [1, 1, 1, 0, None]),
 ('a_p_b', 'a', [1, 1, 1, 0, None]),
 ('a_p_b', 'b', [1, 1, 1, 0, None])]

In [43]:
len(single_permute_list)   # 6

6

In [44]:
spl = pd.DataFrame(single_permute_list, columns = ["colToProject", "intermediaryCol", "cMeasures?"])
spl.shape   # (210, 4)
spl

,colToProject,intermediaryCol,cMeasures?
0,a,b,"[1, 1, 1, 0, None]"
1,a,a_p_b,"[1, 1, 1, 0, None]"
2,b,a,"[1, 1, 1, 0, None]"
3,b,a_p_b,"[1, 1, 1, 0, None]"
4,a_p_b,a,"[1, 1, 1, 0, None]"
5,a_p_b,b,"[1, 1, 1, 0, None]"


In [45]:
X = train

In [46]:
utils.ulevelsPerThreshold(X,'a_p_b', threshold_list = [0.00001, 0.0001, 0.0002, 0.000485,0.0005, 0.001, 0.01, 0.1, 0.15, 0.2])

,apply %threshold,remaining_levels
0,0.000000,184797
1,0.000010,184798
2,0.000100,184798
3,0.000200,184798
4,0.000485,13948
5,0.000500,13948
6,0.001000,3400
7,0.010000,33
8,0.100000,1
9,0.150000,1


In [47]:
mergethreshold = 0.00050  # 0.0020
X =  utils.levelsTransformer(X, 'a_p_b',mergethreshold, mapdictPath)

No saved dictionary exists as yet.


In [50]:
record_sets = list(spl.to_records(index = False))
record_sets

[('a', 'b', list([1, 1, 1, 0, None])),
 ('a', 'a_p_b', list([1, 1, 1, 0, None])),
 ('b', 'a', list([1, 1, 1, 0, None])),
 ('b', 'a_p_b', list([1, 1, 1, 0, None])),
 ('a_p_b', 'a', list([1, 1, 1, 0, None])),
 ('a_p_b', 'b', list([1, 1, 1, 0, None]))]

In [51]:
fulllist = [record_sets]

In [ ]:
# Reduce no of unique values in a_p_b

In [52]:
m = 1
for k,j in enumerate(fulllist):
  print("Record set no is {} of {}th fulllist ".format(k+1,m+1))
  print("Set of records is: ", j)
  # Each outer loop creates a new sets of models
  fulllist_repeat_no = m + 1
  transYesNo = False  # True implies transformation
  X1_train, X1_test = network_features.get_store_features(X,j, transYesNo, None, fulllist_repeat_no , modelsPath, pathToStoreProgress)
  print("\nList done. X1_train shape ", X1_train.shape)
  del X1_train
  gc.collect()

Record set no is 1 of 2th fulllist 
Set of records is:  [('a', 'b', list([1, 1, 1, 0, None])), ('a', 'a_p_b', list([1, 1, 1, 0, None])), ('b', 'a', list([1, 1, 1, 0, None])), ('b', 'a_p_b', list([1, 1, 1, 0, None])), ('a_p_b', 'a', list([1, 1, 1, 0, None])), ('a_p_b', 'b', list([1, 1, 1, 0, None]))]
('a', 'b', list([1, 1, 1, 0, None]))

Next:  1 a b
No of unique values in  a  are:  4044
Bipartite Network created. Time taken:  0.01978335777918498 minutes
Bipartite Network projected. Time taken:  0.34839026133219403 minutes
Model extracted. Time taken:  0.4221627672513326 minutes
Model stored to disk
train columns:  3
No test data
Done with a b
Time taken:  0.8099716424942016
Entry made in file progress.csv
('a', 'a_p_b', list([1, 1, 1, 0, None]))

Next:  2 a a_p_b
No of unique values in  a  are:  4044
Bipartite Network created. Time taken:  0.007215933005015055 minutes
Bipartite Network projected. Time taken:  0.9191853086153666 minutes
Model extracted. Time taken:  1.1277132352193198 m

In [ ]:
# Update levels or merge rarely occurring levels
#  into one. Dictionaries for performing this mapping
#   are in folder: mapdictpath
X = utils.updateLevels(X,
                        mapdictPath,
                        cat_cols
                        )

In [ ]:
def readStoredModels(pathToReadFrom, fileList):
  model_dfs = []
  os.chdir(pathToReadFrom)
  for file in fileList:
    df = pd.read_pickle(file)
    model_dfs.append(df)
  return model_dfs


In [ ]:
os.chdir(modelsPath)
files = os.listdir()
file_list = [file for file in files if ("deg_a_ck" in file) or ("eig_a_ck" in file) or ("pr_a_ck" in file) ]


In [ ]:
models = readStoredModels(modelsPath, file_list)

In [ ]:
# Merge models
for i, model in enumerate(models):
  if i == 0:
    df = model
  else:
    df = pd.merge(df, model, left_index=True, right_index=True)  

In [ ]:
df

In [ ]:
# Save dataframe:
os.chdir(dataPath)
df.to_pickle("word2vec.pkl")


In [ ]:
# Load word2vec
os.chdir(dataPath)
df_word= pd.read_pickle("word2vec.pkl")
df_word.head()

,deg_a_ck_b,eig_a_ck_b,pr_a_ck_b,deg_a_ck_a_p_b,eig_a_ck_a_p_b,pr_a_ck_a_p_b
a,,,,,,
1.0,1.000000,0.075681,0.003534,0.0,-0.059685,0.002083
2.0,0.987474,0.074952,0.003492,0.0,0.077926,0.002083
3.0,0.983299,0.074591,0.003479,0.0,0.010066,0.002083
4.0,0.974948,0.074072,0.003452,0.0,0.007677,0.002083
5.0,0.991649,0.075158,0.003507,0.0,0.061111,0.002083


In [ ]:
os.chdir(dataPath)
df_word.to_csv('vectors.tsv', sep="\t", index = False, header = False)

In [ ]:
metadata = ast.literal_eval(re.search('({.+})', word2index).group(0))

In [ ]:
str_list = list(metadata.keys())
str_list

In [ ]:
import sys
orig_stdout = sys.stdout
f = open('metadata.tsv', 'w')
sys.stdout = f

print(' \n'.join(str_list))

sys.stdout = orig_stdout
f.close()
sys.stdout = orig_stdout